# **Main CITE Notebook**

This is the main notebook for CITE part of the project, where the task is to predict normalized surface protein levels given information about library-size normalized and log1p transformed counts (gene expression levels) for the same cells.

In this Jupyter notebook, data from several sources is joined together and is used further to either cross-validate or create predictions for the test dataset. The sources are:
1. pre-calculated Truncated SVD values from all the RNA expression levels data (see Prepare_SVD_for_CITE notebook)
2. source data for RNA gene expression levels - for pre-selected important genes this data is going to be put into the model as is
3. metadata - cells' donor ID and day the cells were analyzed. Few features are built from metadata information.
4. target values for the train set

In [1]:
# Importing the libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import gc
from humanize import naturalsize
#import lightgbm
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

# **Loading the Data**

In [2]:
DATA_DIR = "/kaggle/input/open-problems-multimodal/"
FP_CELL_METADATA = os.path.join(DATA_DIR,"metadata.csv")

FP_CITE_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_cite_inputs.h5")
FP_CITE_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_cite_targets.h5")
FP_CITE_TEST_INPUTS = os.path.join(DATA_DIR,"test_cite_inputs.h5")
FP_CITE_TEST_INPUTS_FIX = os.path.join(DATA_DIR,"test_cite_inputs_day_2_donor_27678.h5")

FP_MULTIOME_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_multi_inputs.h5")
FP_MULTIOME_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_multi_targets.h5")
FP_MULTIOME_TEST_INPUTS = os.path.join(DATA_DIR,"test_multi_inputs.h5")

FP_SUBMISSION = os.path.join(DATA_DIR,"sample_submission.csv")
FP_EVALUATION_IDS = os.path.join(DATA_DIR,"evaluation_ids.csv")

In [3]:
# Load list of important columns
important_columns_path = '../input/kagglegenes/important_columns.csv'
df_imp_cols =  pd.read_csv(important_columns_path)

In [4]:
%%time

# Load train data. In this case, it is impossible to load only important columns, 
# so I have to load all the data, and then remove columns that are not in important columns list.
df = pd.read_hdf(FP_CITE_TRAIN_INPUTS)
df = df[df_imp_cols['important_columns']]
print('import finished')
gc.collect()
df.shape

import finished
CPU times: user 23.9 s, sys: 7.62 s, total: 31.5 s
Wall time: 52.7 s


(70988, 642)

In [5]:
# Check dataframe size
size = df.memory_usage(deep='True').sum()
print(naturalsize(size))

187.2 MB


In [6]:
# I made it possible to run this notebook with SUBMIT = False in case if I am going to run cross-validation only
SUBMIT = True
CROSS_VALIDATE = False

In [7]:
%%time
# Load the test data (this step is skipped if notebook runs only for cross-validation)

if SUBMIT:
    df_test = pd.read_hdf(FP_CITE_TEST_INPUTS)
    df_test = df_test[df_imp_cols['important_columns']]
    print('import of test data finished')
    print(df_test.shape)
    

import of test data finished
(48663, 642)
CPU times: user 16.3 s, sys: 2.47 s, total: 18.7 s
Wall time: 32.9 s


In [8]:
%%time

# Load metadata and select rows that refer to CITE
md_df = pd.read_csv(FP_CELL_METADATA, index_col='cell_id')
md_df = md_df.loc[md_df['technology'] == "citeseq"]
md_df['day'] = md_df['day'].astype('int8')
del md_df['technology']
# Join the metadata to train.
df = df.join(md_df, how = 'left', on = 'cell_id')
# Join the metadata to test.
if SUBMIT:
    df_test = df_test.join(md_df, how = 'left', on = 'cell_id')
    print(df_test.shape)
del md_df
gc.collect()
df.shape

(48663, 645)
CPU times: user 529 ms, sys: 82 ms, total: 611 ms
Wall time: 730 ms


(70988, 645)

In [9]:
%%time
# Load the prepared TruncatedSVD data and join it to the rest of train/test datasets.
# There was a mess with organizers initially publishing wrong data as part of test set.
# They've later released a fix (see FP_CITE_TEST_INPUTS_FIX) for information only.
# This means participants didn't have to make predictions for the fixed data, but could use it for their models.
# I used that data to calculate the TruncatedSVD.
# This is why this part of code is not as simple as it could be otherwise.

svd_path = '../input/notebookb136a7c42f/svd.csv'
svd = pd.read_csv(svd_path)
svd = svd.add_prefix('svd_')
svd_train = svd.iloc[:70988]
print(svd_train.shape)

#join the svd data to train
df = df.reset_index()
df = pd.concat([df, svd_train], axis=1)

#join the svd data to test
if SUBMIT:
    df_test = df_test.iloc[7476:]
    df_test = df_test.reset_index()
    svd_test = svd.iloc[70988:112175].reset_index(drop = True)
    print(svd_test.shape)
    df_test = pd.concat([df_test, svd_test], axis=1)
    del svd_test

del svd, svd_train
gc.collect()
print(df.shape)
if SUBMIT:
    print(df_test.shape)

(70988, 512)
(41187, 512)
(70988, 1158)
(41187, 1158)
CPU times: user 14.2 s, sys: 1.42 s, total: 15.6 s
Wall time: 27.4 s


In [10]:
%%time
# Import the target values and merge them into train dataframe.

Y = pd.read_hdf(FP_CITE_TRAIN_TARGETS)
Y = Y.add_prefix('y_')
print(f"Y shape: {str(Y.shape):14} {Y.size*4/1024/1024/1024:2.3f} GByte")
df = df.join(Y, how = 'left', on = 'cell_id')

del Y
gc.collect()
df.shape

Y shape: (70988, 140)   0.037 GByte
CPU times: user 419 ms, sys: 123 ms, total: 542 ms
Wall time: 960 ms


(70988, 1298)

In [11]:
'''
# Experimented with some manually crafted features from raw data, but decided not to include them into final submission.
raw_features_path = '../input/values-from-raw/features_from_raw.ftr'
df_raw_features = pd.read_feather(raw_features_path)
df = df.merge(df_raw_features, how = 'left', on = 'cell_id')
del df_raw_features
gc.collect()
print(df.shape)
if SUBMIT:
    test_raw_features_path = '../input/values-from-raw/features_from_raw_test.ftr'
    df_raw_features_test = pd.read_feather(test_raw_features_path)
    df_test = df_test.merge(df_raw_features_test, how = 'left', on = 'cell_id')
    del df_raw_features_test
    gc.collect()
    print(df_test.shape)
'''

"\n# Experimented with some manually crafted features from raw data, but decided not to include them into final submission.\nraw_features_path = '../input/values-from-raw/features_from_raw.ftr'\ndf_raw_features = pd.read_feather(raw_features_path)\ndf = df.merge(df_raw_features, how = 'left', on = 'cell_id')\ndel df_raw_features\ngc.collect()\nprint(df.shape)\nif SUBMIT:\n    test_raw_features_path = '../input/values-from-raw/features_from_raw_test.ftr'\n    df_raw_features_test = pd.read_feather(test_raw_features_path)\n    df_test = df_test.merge(df_raw_features_test, how = 'left', on = 'cell_id')\n    del df_raw_features_test\n    gc.collect()\n    print(df_test.shape)\n"

In [12]:
# Made experiments with using TruncatedSVD built on raw data, but cross-validation showed it only decreases
# model's performance, even if I only take first 32 components.
'''
svd_raw_path = '../input/svd-from-raw-data/svd_raw.csv'
svd_raw = pd.read_csv(svd_raw_path)
svd_raw = svd_raw.add_prefix('ext_')
svd_train_raw = svd_raw.iloc[:70988, :32]
#svd = pd.DataFrame(svd, dtype='float16')
print(svd_train_raw.shape)

#join the svd data to train
df = pd.concat([df, svd_train_raw], axis=1)

#join the svd data to test
if SUBMIT:
    print(df_test.shape)
    svd_test_raw = svd_raw.iloc[70988:112175, :32].reset_index(drop=True)
    print(svd_test_raw.shape)
    df_test = pd.concat([df_test, svd_test_raw], axis=1)
    del svd_test_raw

del svd_raw, svd_train_raw
gc.collect()
print(df.shape)
if SUBMIT:
    print(df_test.shape)
'''

"\nsvd_raw_path = '../input/svd-from-raw-data/svd_raw.csv'\nsvd_raw = pd.read_csv(svd_raw_path)\nsvd_raw = svd_raw.add_prefix('ext_')\nsvd_train_raw = svd_raw.iloc[:70988, :32]\n#svd = pd.DataFrame(svd, dtype='float16')\nprint(svd_train_raw.shape)\n\n#join the svd data to train\ndf = pd.concat([df, svd_train_raw], axis=1)\n\n#join the svd data to test\nif SUBMIT:\n    print(df_test.shape)\n    svd_test_raw = svd_raw.iloc[70988:112175, :32].reset_index(drop=True)\n    print(svd_test_raw.shape)\n    df_test = pd.concat([df_test, svd_test_raw], axis=1)\n    del svd_test_raw\n\ndel svd_raw, svd_train_raw\ngc.collect()\nprint(df.shape)\nif SUBMIT:\n    print(df_test.shape)\n"

# **Cross-validation**

In [13]:
lightgbm_params = {
     'learning_rate': 0.05, 
     'max_depth': 7, 
     'min_child_samples': 100,
     'subsample': 0.6, 
     "seed": 1,
     "device": "gpu",
     "gpu_platform_id": 0,
     "gpu_device_id": 0,
    }

In [14]:
# Function to calculate the competition's custom metric.

def correlation_score(y_true, y_pred):
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)

In [15]:
if CROSS_VALIDATE:
    df['svd_var1'] = 0
    #df['svd_var2'] = 0
    df['svd_var3'] = 0
    df['svd_var4'] = 0
    df['svd_sex'] = 0
    df.loc[df['donor'] == 13176, 'svd_sex'] = 1
    
    x_cols = [col for col in list(df.columns) if (col.startswith('ENSG')) | (col.startswith('svd'))]
    y_cols = [col for col in list(df.columns) if (col.startswith('y_'))]

In [16]:
# At some point, close to competition end decided to use different catboost parameters depending on target.
cat_params_slow = {
    "random_state": 23,
    "learning_rate" : 0.02,
    "eval_metric" : 'RMSE', 
    "max_depth" : 7,
    "verbose" : 100,
    "n_estimators" : 300,
    "task_type" : 'GPU'
    }
cat_params_middle = {
    "random_state": 23,
    "learning_rate" : 0.03,
    "eval_metric" : 'RMSE', 
    "max_depth" : 7,
    "verbose" : 100,
    "n_estimators" : 400,
    "task_type" : 'GPU'
    }
cat_params_fast = {
    "random_state": 23,
    "learning_rate" : 0.05,
    "eval_metric" : 'RMSE', 
    "max_depth" : 7,
    "verbose" : 100,
    "n_estimators" : 500,
    "task_type" : 'GPU'
    }

In [17]:
# Define the targets' grouping depending on how well they can be predicted.
df_model_groups = pd.read_csv('../input/model-groups/model_groups.csv')
df_model_groups['mean'] = df_model_groups.mean(axis=1, numeric_only=True)
df_model_groups['group'] = 2
df_model_groups.loc[df_model_groups['mean'] < 0.25, 'group'] = 1
df_model_groups.loc[df_model_groups['mean'] > 0.5 , 'group'] = 3

y_slow = list(df_model_groups.loc[df_model_groups['group'] == 1, 'columns'])
y_fast = list(df_model_groups.loc[df_model_groups['group'] == 3, 'columns'])

In [18]:
# Used this to run cross-validation only on 5 targets of 140 - for test.
#y_cols = y_cols[:5]

# Function to perform cv split by day or by donor.
# Closer to the end of competition, cross-validated only on days.
# Also added cross-validation with evaluation set to check how well models converge.
def custom_cv_split(option, d_frame, USE_EVAL_SET=False):
    d_frame['svd_var1'] = d_frame['day']
    if option > 20:
        cv_col = 'donor'
    else:
        d_frame.loc[d_frame['donor'] == 31800, 'svd_var3'] = 1
        d_frame.loc[d_frame['donor'] == 32606, 'svd_var4'] = 1
        cv_col = 'day'
    Xc_tr = d_frame.loc[d_frame[cv_col] != option, x_cols]
    Xc_va = d_frame.loc[d_frame[cv_col] == option, x_cols]
    yc_va = d_frame.loc[d_frame[cv_col] == option, y_cols]
    if USE_EVAL_SET:
        Xc_tr_cv = Xc_tr.sample(frac=0.1, random_state=25)
        Xc_tr = Xc_tr.drop(Xc_tr_cv.index)
        yc_tr_cv = d_frame[y_cols].iloc[Xc_tr_cv.index]
        yc_tr = d_frame[y_cols].iloc[Xc_tr.index]
        return Xc_tr, yc_tr, Xc_va, yc_va, Xc_tr_cv, yc_tr_cv
    else:
        yc_tr = d_frame[y_cols].iloc[Xc_tr.index]
        return Xc_tr, yc_tr, Xc_va, yc_va

In [19]:
%%time
# Cross-validation with LGBMRegressor or CatboostRegressor in a loop.

if CROSS_VALIDATE:
    df_mse_corr = pd.DataFrame({'columns': y_cols})
    df_importances = pd.DataFrame({'columns':x_cols})
    #cv_options = [2,3,4, 32606, 31800]
    cv_options = [2,3,4]
    score_list = []
    for cv_option in cv_options:
        X_tr, y_tr, X_va, y_va = custom_cv_split(cv_option, df)
        mse_list = []
        corr_list = []
        for i in range(len(y_cols)):
            print(str(cv_option) + '_' + str(i) + '_' + y_cols[i])
            column_name = str(cv_option) + '_' + y_cols[i]
            y_column_name = 'pred_' + y_cols[i]
            #model = lightgbm.LGBMRegressor(**lightgbm_params)
            if y_cols[i] in y_slow:
                model = CatBoostRegressor(**cat_params_slow)
            elif y_cols[i] in y_fast:
                model = CatBoostRegressor(**cat_params_fast)
            else:
                model = CatBoostRegressor(**cat_params_middle)
            model.fit(X_tr, y_tr.iloc[:,i].copy())
            y_va[y_column_name] = model.predict(X_va)
            df_importances[column_name] = model.feature_importances_
            mse_value = mean_squared_error(y_va.iloc[:,i], y_va.iloc[:,i+len(y_cols)], squared=False)
            corr_value = np.corrcoef(y_va.iloc[:,i], y_va.iloc[:,i+len(y_cols)])[1, 0]
            mse_list.append(mse_value)
            corr_list.append(corr_value)
            print(str(mse_value) + '_' + str(corr_value))
        gc.collect()
        
        # Save the metric values for all 140 models
        cv_option_corr = str(cv_option) + '_corr'
        cv_option_mse = str(cv_option) + '_mse'
        df_mse_corr[cv_option_corr] = corr_list
        df_mse_corr[cv_option_mse] = mse_list   

        # Validate the model (mse and correlation over all 140 columns)
        y_preds = ['pred_' + y for y in y_cols]
        mse = mean_squared_error(y_va[y_cols], y_va[y_preds])
        corrscore = correlation_score(y_va[y_cols], y_va[y_preds])         

        print(f"Fold {cv_option}: mse = {mse:.5f}, corr =  {corrscore:.5f}")

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


In [20]:
# Export the cross-validation predictions for a second-level model.
# Wasn't used in final submission.

if CROSS_VALIDATE & SUBMIT:
    for cols in df_cv_pred:
        if col in y_cols:
            del df_cv_pred[col]
    gc.collect()
    df_cv_pred = df_cv_pred.reset_index()
    export_file_results = 'results_cite_cv.ftr'
    df_cv_pred.to_feather(export_file_results)
    del df_cv_pred


In [21]:
if CROSS_VALIDATE:
    for col in ['svd_var1', 'svd_var2', 'svd_var3', 'svd_var4', 'svd_sex']:
        if col in df.columns:
            del df[col]

gc.collect()

42

In [22]:
# Save the feature importance data.

if CROSS_VALIDATE & SUBMIT:
    export_file_importances = 'imps_cite.ftr'
    df_importances.to_feather(export_file_importances)
    del df_importances

gc.collect()

21

# **Prediction**

In [23]:
# Create features from metadata.
# Note: I couldn't use get_dummies function here, as one of days and one of donors only appear in test dataset.

def add_metadata_features(d_frame):
    d_frame['svd_x_donor_13176'] = 0
    d_frame['svd_x_donor_31800'] = 0
    d_frame['svd_x_donor_32606'] = 0
    d_frame.loc[d_frame['donor'] == 13176, 'svd_x_donor_13176'] = 1
    d_frame.loc[d_frame['donor'] == 31800, 'svd_x_donor_31800'] = 1
    d_frame.loc[d_frame['donor'] == 32606, 'svd_x_donor_32606'] = 1
    d_frame['svd_x_day'] = d_frame['day']
    return d_frame


In [24]:
# Predict. Ran a few time with cross-validation on 10% of data to check if it makes sense to add more
# iterations. 
'''
if SUBMIT:
    df = add_metadata_features(df)
    df_test = add_metadata_features(df_test)
    x_cols = [col for col in list(df.columns) if (col.startswith('ENSG')) | (col.startswith('svd'))]
    y_cols = [col for col in list(df.columns) if (col.startswith('y_'))]
    best_iteration = []
    best_cv_result = []
    #y_cols = y_cols[:5]
    df_cv = df.sample(frac=0.1, random_state=25)
    df= df.drop(df_cv.index)
    X = df[x_cols].values
    Y = df[y_cols].values
    X_cv = df_cv[x_cols].values
    Y_cv = df_cv[y_cols].values
    for i in range(len(y_cols)):
        print('Training_column: ' + str(i))
        #model = lightgbm.LGBMRegressor(n_estimators=n_estimators, **lightgbm_params)
        #model = CatBoostRegressor(n_estimators=n_estimators, **cat_params)
        if y_cols[i] in y_slow:
            model = CatBoostRegressor(**cat_params_slow)
        elif y_cols[i] in y_fast:
            model = CatBoostRegressor(**cat_params_fast)
        else:
            model = CatBoostRegressor(**cat_params_middle)
        model.fit(X, Y[:,i].copy(), eval_set=[(X_cv, Y_cv[:,i].copy())])
        best_iteration.append(model.best_iteration_)
        best_cv_result.append(correlation_value)
        correlation_value = np.corrcoef(model.predict(X_cv),  Y_cv[:,i])[1, 0]
        print(str(correlation_value))
'''

"\nif SUBMIT:\n    df = add_metadata_features(df)\n    df_test = add_metadata_features(df_test)\n    x_cols = [col for col in list(df.columns) if (col.startswith('ENSG')) | (col.startswith('svd'))]\n    y_cols = [col for col in list(df.columns) if (col.startswith('y_'))]\n    best_iteration = []\n    best_cv_result = []\n    #y_cols = y_cols[:5]\n    df_cv = df.sample(frac=0.1, random_state=25)\n    df= df.drop(df_cv.index)\n    X = df[x_cols].values\n    Y = df[y_cols].values\n    X_cv = df_cv[x_cols].values\n    Y_cv = df_cv[y_cols].values\n    for i in range(len(y_cols)):\n        print('Training_column: ' + str(i))\n        #model = lightgbm.LGBMRegressor(n_estimators=n_estimators, **lightgbm_params)\n        #model = CatBoostRegressor(n_estimators=n_estimators, **cat_params)\n        if y_cols[i] in y_slow:\n            model = CatBoostRegressor(**cat_params_slow)\n        elif y_cols[i] in y_fast:\n            model = CatBoostRegressor(**cat_params_fast)\n        else:\n         

In [25]:
# Make predictions.
if SUBMIT:
    df = add_metadata_features(df)
    df_test = add_metadata_features(df_test)
    x_cols = [col for col in list(df.columns) if (col.startswith('ENSG')) | (col.startswith('svd'))]
    y_cols = [col for col in list(df.columns) if (col.startswith('y_'))]
    #y_cols = y_cols[:5]
    X = df[x_cols].values
    Y = df[y_cols].values
    Xt = df_test[x_cols].values
    for i in range(len(y_cols)):
        print('Training_column: ' + str(i))
        #model = lightgbm.LGBMRegressor(n_estimators=n_estimators, **lightgbm_params)
        if y_cols[i] in y_slow:
            model = CatBoostRegressor(**cat_params_slow)
        elif y_cols[i] in y_fast:
            model = CatBoostRegressor(**cat_params_fast)
        else:
            model = CatBoostRegressor(**cat_params_middle)
        model.fit(X, Y[:,i].copy())
        col_name = y_cols[i]
        df_test[col_name] = model.predict(Xt)


Training_column: 0
0:	learn: 1.3880658	total: 37.9ms	remaining: 15.1s
100:	learn: 1.2274369	total: 2.89s	remaining: 8.56s
200:	learn: 1.2000120	total: 5.27s	remaining: 5.22s
300:	learn: 1.1826643	total: 7.69s	remaining: 2.53s
399:	learn: 1.1691044	total: 10.1s	remaining: 0us
Training_column: 1
0:	learn: 0.8930985	total: 28.2ms	remaining: 8.44s
100:	learn: 0.8547100	total: 2.59s	remaining: 5.1s
200:	learn: 0.8490743	total: 5.46s	remaining: 2.69s
299:	learn: 0.8451139	total: 7.99s	remaining: 0us
Training_column: 2
0:	learn: 0.9192535	total: 29.1ms	remaining: 11.6s
100:	learn: 0.8435130	total: 2.95s	remaining: 8.74s
200:	learn: 0.8361452	total: 5.53s	remaining: 5.47s
300:	learn: 0.8302002	total: 8.13s	remaining: 2.67s
399:	learn: 0.8246224	total: 10.7s	remaining: 0us
Training_column: 3
0:	learn: 2.4953868	total: 27.9ms	remaining: 13.9s
100:	learn: 1.7882656	total: 2.69s	remaining: 10.6s
200:	learn: 1.7434499	total: 5.61s	remaining: 8.34s
300:	learn: 1.7154959	total: 8.19s	remaining: 5.42s

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


Training_column: 96
0:	learn: 1.0488591	total: 26.2ms	remaining: 10.5s
100:	learn: 0.9616378	total: 2.5s	remaining: 7.4s
200:	learn: 0.9527839	total: 4.93s	remaining: 4.88s
300:	learn: 0.9465803	total: 7.47s	remaining: 2.46s
399:	learn: 0.9406954	total: 10.1s	remaining: 0us
Training_column: 97
0:	learn: 2.6639259	total: 30.5ms	remaining: 15.2s
100:	learn: 1.7158407	total: 2.99s	remaining: 11.8s
200:	learn: 1.6561376	total: 5.58s	remaining: 8.3s
300:	learn: 1.6281572	total: 8.11s	remaining: 5.36s
400:	learn: 1.6067617	total: 10.6s	remaining: 2.61s
499:	learn: 1.5870562	total: 14s	remaining: 0us
Training_column: 98
0:	learn: 0.8464169	total: 27.4ms	remaining: 8.19s
100:	learn: 0.8191111	total: 2.6s	remaining: 5.12s
200:	learn: 0.8143471	total: 5.58s	remaining: 2.75s
299:	learn: 0.8106361	total: 8.16s	remaining: 0us
Training_column: 99
0:	learn: 4.0724575	total: 35.7ms	remaining: 17.8s
100:	learn: 2.4278199	total: 2.75s	remaining: 10.9s
200:	learn: 2.3398703	total: 5.36s	remaining: 7.97s


In [26]:
if SUBMIT:
    del df, X, Y, Xt
    for col in df_test.columns:
        if col in x_cols:
            del df_test[col]
    gc.collect()

In [27]:
# Export predictions to file.
if SUBMIT:
    export_file_results = 'results_cite.ftr'
    df_test.to_feather(export_file_results)